In [19]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.image as mpimg
import glob



In [20]:
def read_data():
	input_image = []
	i = 0
	for filename in glob.iglob('../data/tiny-imagenet-200/train/**/*.JPEG',recursive=True):
		img = mpimg.imread(filename)
		if(img.shape == (64,64,3)):
			input_image.append(list(mpimg.imread(filename)))
		if(i%10000 == 0):
			print(i)
		i+=1
		#if(i>5000):
		#	break

	return np.asarray(input_image)

In [21]:
image_data = read_data()
print (image_data.shape)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
(98179, 64, 64, 3)


In [22]:
### Converting RGB TO GRAYSCALE DEMO MODEL

def simple_rgb_to_gray(X,Y):
	Wconv1 = tf.get_variable("Wconv1", shape=[1, 1, 3, 1])
	bconv1 = tf.get_variable("bconv1", shape=[1])
	y_out = tf.nn.conv2d(X, Wconv1, strides=[1,1,1,1], padding='SAME')
	y_out = y_out[:,:,:,0]
	return y_out

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])


In [23]:
X = tf.placeholder(tf.float32, [None, 64, 64, 3])
Y = tf.placeholder(tf.float32 ,[None , 64,64])
y_out = simple_rgb_to_gray(X,Y)
loss = tf.nn.l2_loss(y_out - Y)
optimiser = tf.train.AdamOptimizer(0.4)
train_step = optimiser.minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for i in range(600):
    
	feed_dict = {X: image_data[i*100:100*i+100], Y: rgb2gray(image_data[i*100:i*100+100])}
	losses,_ = sess.run([loss,train_step],feed_dict)
	if(i%50 ==0):
		print (losses)

3.81945e+08
7.91294e+06
261196.0
2890.45
40.8482
1.20239
0.00367048
0.000115009
8.75046e-06
8.03367e-06
5.72398e-06
7.91956e-06


In [24]:
var  = tf.trainable_variables()
var = [v.name for v in tf.trainable_variables() ]
print (var)
numpy_val = sess.run([var[0]])
weights = np.asarray(numpy_val[0])
print (weights)

print ("Observe that weights are close to [0.299,0.587,0.114] , hence it learns rgb to gray conversion")

['Wconv1:0', 'bconv1:0']
[[[[ 0.29899999]
   [ 0.58700001]
   [ 0.11399999]]]]
Observe that weights are close to [0.299,0.587,0.114] , hence it learns rgb to gray conversion
